In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec
from datetime import datetime
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import copy
from datetime import datetime
import random

#Lectura de identificadores
#Como los archivos descargados siguen un formato que involucra el identificador, es util tener a mano estos datos para agrupar los datos

f = open("Datos/ID_Estaciones.txt", "r")
ids = []
lines = f.readlines()
ids_dict = {}

for line in lines:
    ids.append(line.split(",")[2].split("\n")[0])
    ids_dict[line.split(",")[2].split("\n")[0]] = {}

print(ids)
print(ids_dict)
labels = copy.deepcopy(ids)

['510005', '400009', '180005', '330020', '550001']
{'510005': {}, '400009': {}, '180005': {}, '330020': {}, '550001': {}}


In [108]:
bigDf = pd.DataFrame()
for id in ids:
    df = pd.read_excel(id+"_agrupado.xlsx", sheet_name="Sheet1")
    if(bigDf.size == 0):
        bigDf = pd.concat([df.iloc[:,1:]], ignore_index = True)
    else:
        bigDf = pd.concat([bigDf,df.iloc[:,1:]], ignore_index = True)

bigDf["momento"] = pd.to_datetime(bigDf["momento"], format="%d-%m-%Y %H:%M:%S")
bigDf.dropna()

,ID_Estacion,momento,RR6_Valor,HR_Valor,Ts_Valor,ff_Valor,RadGInst (watt/m2)
0,510005,2023-01-01 00:00:00,0.0,57.8,9.7,23.0,78.1
6,510005,2023-01-01 06:00:00,0.0,63.8,7.9,17.0,0.0
12,510005,2023-01-01 12:00:00,0.2,60.0,10.0,13.0,290.5
18,510005,2023-01-01 18:00:00,0.0,42.0,15.0,31.0,1085.2
24,510005,2023-01-02 00:00:00,0.0,52.9,12.4,22.0,70.8
...,...,...,...,...,...,...,...
43216,550001,2023-12-30 18:00:00,0.0,59.5,11.4,12.0,222.1
43222,550001,2023-12-31 00:00:00,0.0,62.7,10.8,15.0,128.5
43228,550001,2023-12-31 06:00:00,0.0,78.5,8.2,4.0,0.0
43234,550001,2023-12-31 12:00:00,0.0,40.0,12.0,9.0,431.0


In [109]:
#Boxplotter(visualizacion de outliers o anomalias en la velocidad del viento)
def boxPlot_viento():

    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    mesIndex = 0
    estacion = ids[0]
    scale = True
    
    scaleChk = widgets.Checkbox(description="Escala?", value=scale)
    prevB = widgets.Button(description="Anterior")
    nextB = widgets.Button(description="Siguiente")
    mesDrop = widgets.Dropdown(
        options=meses,
        value='Ene',
        description='Mes:',
        disabled=False,
    )
    estacionDrop = widgets.Dropdown(
        options=ids,
        value=ids[0],
        description='Estacion:',
        disabled=False,
    )
    
    output = widgets.Output()
    def changed(change):
        nonlocal scale
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            scale = not scale
            replot(mesIndex,estacion, scale)
    
    def prev_click(b):
        nonlocal mesIndex, scale
        if(mesIndex == 11):
            nextB.disabled = False
        mesIndex = mesIndex - 1
        if(mesIndex <= 0):
            mesIndex = 0
            prevB.disabled = True
        mesDrop.value = meses[mesIndex]
        replot(mesIndex, estacion, scale)

    def next_click(b):
        nonlocal mesIndex, scale
        if(mesIndex == 0):
            prevB.disabled = False
        mesIndex = mesIndex + 1
        if(mesIndex >= 11):
            mesIndex = 11
            nextB.disabled = True
        mesDrop.value = meses[mesIndex]
        replot(mesIndex, estacion, scale)
    
    def meses_dropdown(change):
        nonlocal mesIndex, scale
        if change['type'] == 'change' and change['name'] == 'value':
            mesIndex = meses.index(change['new'])
            if(meses.index(change['new']) == 0):
                prevB.disabled = True
                nextB.disabled = False
            elif(meses.index(change['new']) == 11):
                prevB.disabled = False
                nextB.disabled = True
            else:
                prevB.disabled = False
                nextB.disabled = False
            replot(mesIndex,estacion, scale)
            print("changed to"+ change['new'] +" which is on index"+str(meses.index(change['new'])))

    def estacion_dropdown(change):
        nonlocal estacion, scale
        if(change['type'] == 'change' and change['name'] == 'value'):
            estacion = change['new']
            replot(mesIndex,estacion, scale)
            print("changed to "+ change['new'] +" which is on index "+str(ids.index(change['new'])))
    
    def replot(mes, estacion, scale):
        with output:
            output.clear_output(wait=True)
            fig = plt.figure(figsize=(8,4))
            gs = gridspec.GridSpec(3, 4, wspace=0.50)
            axs = []
            axs.append(fig.add_subplot(gs[:,0]))
            axs.append(fig.add_subplot(gs[:,1:3]))
            axs.append(fig.add_subplot(gs[:,3]))
            estDf = bigDf.query('ID_Estacion == '+estacion)
            
            est = estDf[estDf['momento'].dt.month == mes+1]
            rawData = axs[1].boxplot(est["ff_Valor"].dropna())

            #plt.show()
            if(mes==0):
                est3 = estDf[estDf['momento'].dt.month == mes+2]
                axs[0].get_yaxis().set_visible(False)
                axs[2].boxplot(est3["ff_Valor"].dropna())
                axs[2].set_title(meses[mes+1])
            elif(mes==11):
                est2 = estDf[estDf['momento'].dt.month == mes]
                axs[0].boxplot(est2["ff_Valor"].dropna())
                axs[0].set_title(meses[mes-1])
                axs[2].get_yaxis().set_visible(False)
            else:
                est2 = estDf[estDf['momento'].dt.month == mes]
                est3 = estDf[estDf['momento'].dt.month == mes+2]
                axs[0].boxplot(est2["ff_Valor"].dropna())
                axs[0].set_title(meses[mes-1])
                axs[2].boxplot(est3["ff_Valor"].dropna())
                axs[2].set_title(meses[mes+1])
            for x in range(len(axs)):
                if(scale):
                    axs[x].set_ylim(0,40)
                axs[x].set_ylabel("Velocidad del viento(kt)")
                axs[x].get_xaxis().set_visible(False)
                axs[x].grid()
            axs[1].set_title("Comportamiento del viento - Año 2023\nEstacion "+estacion+" - Periodo "+meses[mes])
            plt.show()

    replot(mesIndex,estacion,scale)
    nextB.on_click(next_click)
    prevB.on_click(prev_click)
    scaleChk.observe(changed)
    mesDrop.observe(meses_dropdown)
    estacionDrop.observe(estacion_dropdown)
    
    H = widgets.HBox([prevB,mesDrop,nextB, scaleChk], layout={'width':'80%'})
    V_in = widgets.VBox([output,H,estacionDrop], layout={'border': '1px solid black'})
    
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de caja de velocidad del viento mensual:
        A pesar de que ya se ha realizado una visualizacion con la velocidad del viento, esta permite visualizar el comportamiento general y las anomalias en la velocidad del viento con más detalle de una estacion determinada durante un mes y el mes previo/siguiente.
        Con esta visualizacion uno puede hacerse una idea de los periodos en los cuales se puede aprovechar de forma consistente un aerogenerador en un sector determinado.
        En caso de que hubieran demasiadas anomalias en la velocidad del viento, la generacion de electricidad de un aerogenerador seria demasiado inconsistente como para poder ser utilizado con efectividad.
        ''')
    V = widgets.VBox([V_in, textOut], layout={'width':'33%'})

    return V
    #display(V,textOut)

def scatter_vientoHum():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    colours = {}
    s_ids = ids
    s_opt = meses[0]
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()

    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[0],
        description='Periodo:',
        disabled=False,
        layout={'width':'90%'},
        style={'description_width': '30%'}
    )
    
    def meses_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal s_ids, s_opt
            s_opt = change['new']
            #est = estDf[estDf['momento'].dt.month == mes+1]
            replot(s_ids, s_opt)
    
    
    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            #display(sel_data)
            mesIndex = meses.index(option)
            month_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
            for id in selected_ids:
                filtered_df = month_df.query('ID_Estacion == '+id)
                plt.scatter(filtered_df["ff_Valor"], filtered_df["HR_Valor"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
            plt.xlim(0,50)
            plt.ylim(0,100)
            plt.xlabel("Velocidad del viento(kt)")
            plt.ylabel("Humedad relativa del aire(%)")
            plt.suptitle("Relacion de niveles de Humedad y viento - Periodo "+option+"\nAño 2023")
            plt.legend()
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    mesDrop.observe(meses_dropdown)

    replot(s_ids, s_opt)
    
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([accordion, mesDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de humedad y velocidad del viento a nivel mensual:
        Permite ver una relacion entre el viento y los nieveles de humedad en el aire. Se pueden ver diferentes patrones provenientes de diferentes estaciones.
        El viento lleva consigo masas de aire que pueden contener niveles de humedad variable. Esto puede impactar tanto en paneles solares como en aerogeneradores de diferentes formas:
        - Condensacion del agua en la superficie de los paneles solares, disminuyendo la energia generada.
        - Altos niveles de humedad afectan en la densidad del aire, disminuyendo la energia generada por un aerogenerador.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(V)
    return V

#dim_lluvia()
#scatter_vientoHum()

def plotTimeSlider_viento():
    #Velocidad del viento promedio del año, medida por mes.
    global labels
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    start_date = min(bigDf['momento'])
    end_date = max(bigDf['momento'])
    s_d = start_date
    e_d = end_date
    selected = copy.deepcopy(ids)

    lineOutput = widgets.Output()

    dates = pd.date_range(start_date, end_date, freq='h')
    options = [(date.strftime('%d/%m/%Y'), date) for date in dates]
    index = (0, len(options)-1)
    
    selection_range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=index,
        description='Fechas',
        orientation='horizontal',
        layout={'width': '80%'}
    )

    def replot(labels, start, end):
        with lineOutput:
            #display(date_range)
            lineOutput.clear_output(wait=True)
            plt.figure(figsize=(10,4))
            filtered_df = bigDf.loc[(bigDf['momento'] >= start.strftime('%Y-%m-%d')) & (bigDf['momento'] < end.strftime('%Y-%m-%d'))]
            temp = pd.DataFrame(columns=ids)
            for id in labels:
                est = filtered_df.query('ID_Estacion == '+id)
                plt.plot(est['momento'], est['ff_Valor'], color=colours[id], label = id,linewidth=0.75)
            plt.ylabel("Velocidad del viento (kt)")
            plt.xticks(rotation=90)
            plt.legend()
            plt.suptitle("Linea de tiempo de velocidad del viento\nAño 2023")
            plt.grid()
            plt.show()
    
    def set_dateRange(date_range):
        nonlocal s_d, e_d
        s_d = date_range["new"][0]
        e_d = date_range["new"][1]
        replot(labels, s_d, e_d)
    
    def checkboxes(_):
        nonlocal s_d, e_d, selected
        selected = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(selected, s_d, e_d)
        
    replot(labels, s_d, e_d)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    
    selection_range_slider.observe(set_dateRange, 'value')
    
    V_in = widgets.VBox(chkbxs, layout={'width':'80%'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'20%'})
    H = widgets.HBox([lineOutput, ac_chk])
    V_in2 = widgets.VBox([H,selection_range_slider], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea de velocidad del viento en relacion al tiempo:
        Permite ver el comportamiento de la velocidad del viento a lo largo del tiempo de cada estacion. Es posible ajustar el rango de tiempo a visualizar del grafico.
        La escala se ajustara automaticamente en base a los datos visualizados. Esto se debe a que hay pocos datos cuyos valores estan muy fuera de lo normal, causando que no pueda distinguirse bien el comportamiento del resto de las lineas.
        La idea de este grafico es poder ver el comportamiento de la velocidad del viento a lo largo del año de todas las estaciones y comparar entre diferentes estaciones cual es el sector mas conveniente para instalar un aerogenerador.
        ''')
    V = widgets.VBox([V_in2, textOut], layout={'width':'33%'})

    return V
    #display(V)

In [110]:
def scatter_tempRadiacion():
    colours = {}
    s_ids = ids
    s_opt = "Ambos"
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        colours[id+"_line"] = (colours[id+"_scatter"][2],colours[id+"_scatter"][1],colours[id+"_scatter"][0])
    opcDrop = widgets.Dropdown(
        options=["Ambos","Solo Regresion Polinomial", "Solo Dispersion"],
        value='Ambos',
        description='Visualizar',
        disabled=False,
        layout={'width':'90%'}
    )
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()

    def opcDropChange(change):
        if(change['type'] == 'change' and change['name'] == 'value'):
            nonlocal s_ids, s_opt
            s_opt = change['new']
            replot(s_ids, s_opt)

    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,6))
            #display(sel_data)
            for id in selected_ids:
                filtered_df = bigDf.query('ID_Estacion == '+id)
                if(option == 'Solo Dispersion' or option == 'Ambos'):
                    plt.scatter(filtered_df["Ts_Valor"], filtered_df["RadGInst (watt/m2)"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
                if(option == 'Solo Regresion Polinomial' or option == 'Ambos'):
                    lrDf = pd.DataFrame()
                    lrDf["x"] = filtered_df["Ts_Valor"]
                    lrDf["y"] = filtered_df["RadGInst (watt/m2)"]
                    lrDf = lrDf.dropna(axis=0)
                    X_axis=pd.DataFrame(lrDf["x"])
                    Y_axis=pd.DataFrame(lrDf["y"])
                    poly = PolynomialFeatures(degree=3, include_bias=False)
                    poly_features = poly.fit_transform(X_axis.values.reshape(-1, 1))
                    lmDict[id] = LinearRegression()
                    models = lmDict[id].fit(poly_features, Y_axis)
                    plt.plot(X_axis,models.predict(poly_features),color=colours[id+"_line"], label = id+"_RP",linewidth=1.5)
            plt.xlim(-20,35)
            plt.ylim(0,1400)
            plt.xlabel("Temperatura(C°)")
            plt.ylabel("Radiacion Global Instantanea(watt/m2)")
            plt.suptitle("Relacion de niveles de Radiacion y temperatura\nAño 2023")
            plt.legend()
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    opcDrop.observe(opcDropChange)
    replot(s_ids, s_opt)
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([accordion] + [opcDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput,V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de temperatura y radiacion + Regresion Polinomial:
        Permite visualizar la relacion entre la temperatura y la radiacion medida por cada estacion. Incluye una recta generada por un modelo de regresion polinomial de los datos recolectados de cada estacion.
        La idea de este grafico era ver si existia alguna relacion entre radiacion y temperatura, motivo por el cual se utilizo un modelo de regresion polinomial para experimentar. Previamente se habia utilizado un modelo de regresion lineal.
        El modelo no es muy preciso.
        ''')

    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    return V #display(H,textOut)

    
def plotTimeSlider_rad():
    #Velocidad del viento promedio del año, medida por mes.
    global labels
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    start_date = min(bigDf['momento'])
    end_date = max(bigDf['momento'])
    s_d = start_date
    e_d = end_date
    selected = copy.deepcopy(ids)

    lineOutput = widgets.Output()

    dates = pd.date_range(start_date, end_date, freq='h')
    options = [(date.strftime('%d/%m/%Y'), date) for date in dates]
    index = (0, len(options)-1)
    
    selection_range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=index,
        description='Fechas',
        orientation='horizontal',
        layout={'width': '80%'}
    )

    def replot(labels, start, end):
        with lineOutput:
            #display(date_range)
            lineOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            filtered_df = bigDf.loc[(bigDf['momento'] >= start.strftime('%Y-%m-%d')) & (bigDf['momento'] < end.strftime('%Y-%m-%d'))]
            temp = pd.DataFrame(columns=ids)
            for id in labels:
                est = filtered_df.query('ID_Estacion == '+id).dropna()
                plt.plot(est['momento'], est['RadGInst (watt/m2)'], color=colours[id], label = id,linewidth=0.75)
            plt.ylabel("Radiacion global instantanea (watt/m2)")
            plt.xticks(rotation=90)
            plt.legend()
            plt.suptitle("Linea de tiempo de radiacion instantanea\nAño 2023")
            plt.grid()
            plt.show()
    
    def set_dateRange(date_range):
        nonlocal s_d, e_d, selected
        s_d = date_range["new"][0]
        e_d = date_range["new"][1]
        replot(selected, s_d, e_d)
    
    def checkboxes(_):
        nonlocal s_d, e_d, selected
        selected = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(selected, s_d, e_d)
        
    replot(labels, s_d, e_d)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    
    selection_range_slider.observe(set_dateRange, 'value')
    
    V_in = widgets.VBox(chkbxs, layout={'width':'80%'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'20%'})
    H = widgets.HBox([lineOutput,ac_chk])
    V_in2 = widgets.VBox([H,selection_range_slider], layout={'border': '1px solid black'})
    

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea de radiacion instantanea en relacion al tiempo:
        Nos permite ver los niveles de radiacion registrados a lo largo del año por cada estacion. Se pueden ver los periodos de niveles de radiacion maxima y minima en cada estacion son consistentes entre cada estacion. Tambien puede verse a grandes rasgos, cuales estaciones registran mayores niveles de radiacion entre si.''')

    V = widgets.VBox([V_in2,textOut], layout={'width':'33%'})
    #display(V,textOut)
    return V

def pie_radiacionTotal():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
    mesIndex = 0
    radiacionTotal = {}
    colours = {}
    for i in range(len(labels)):
        radiacionTotal[labels[i]] = (bigDf.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
        colours[labels[i]] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    selected = copy.deepcopy(labels)
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in radiacionTotal.keys()]
    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[len(meses) - 1],
        description='Periodo:',
        disabled=False,
        layout={'width':'90%'}
    )
    s_data = radiacionTotal.keys()
    s_vals = radiacionTotal.values()
    s_tag = meses[len(meses) - 1]
    scatterOutput = widgets.Output()
        
    def my_fmt(x):
        return '{:.4f}%\n({:.0f} watt/m2)'.format(x, sum(radiacionTotal.values())*x/100)
        
    def meses_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal s_data, s_vals, s_tag, radiacionTotal
            if(change['new'] == 'Anual'):
                for i in range(len(labels)):
                    radiacionTotal[labels[i]] = (bigDf.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
            else:
                mesIndex = meses.index(change['new'])
                f_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
                for i in range(len(labels)):
                    radiacionTotal[labels[i]] = (f_df.query('ID_Estacion == '+labels[i])["RadGInst (watt/m2)"].sum())
            s_data = [label for label,chk in zip(radiacionTotal.keys(),chkbxs) if chk.value]
            s_vals = [radiacionTotal[label] for label in s_data]
            s_tag = change['new']
            #est = estDf[estDf['momento'].dt.month == mes+1]
            replot(s_data, s_vals, s_tag)
    
    def replot(data, values, tag):
        nonlocal colours
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,6))
            plt.pie(values, colors=[colours[key] for key in data], labels=data,autopct=my_fmt)
            plt.legend(title = "ID Estacion")
            plt.title("Radiacion Global Instantanea acumulada - Año 2023\nPeriodo "+tag)
            ax = plt.gca()
            handles, labels = ax.get_legend_handles_labels()
            patch = mpatches.Patch(color='white', label="Total acumulado(visualizado):\n"+str(sum(values))+" watt/m2")
            handles.append(patch) 
            plt.legend(handles=handles, loc='upper right')
            plt.show()
    
    def checkboxes(_):
        nonlocal s_data, s_vals, s_tag
        s_data = [label for label,chk in zip(radiacionTotal.keys(),chkbxs) if chk.value]
        s_vals = [radiacionTotal[label] for label in s_data]
        replot(s_data,s_vals,s_tag)

    replot(radiacionTotal.keys(), radiacionTotal.values(), meses[len(meses) - 1])
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')
    mesDrop.observe(meses_dropdown)

    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    V_in2 = widgets.VBox([ac_chk, mesDrop], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, V_in2], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de torta de la radiacion acumulada durante un periodo:
        Permite visualizar la radiacion acumulada que ha sido registrada por todas las estaciones y el porcentaje que aporta cada una, durante un periodo de un mes determinado o durante el año.
        La idea de este grafico es poder determinar solo por la cantidad de radiacion que ha sido registrada, si es que es conveniente utilizar paneles solares en un determinado sector, segun la estacion que este asignada a este.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    
    return V    #display(H,textOut)


#dim_lluvia()

In [111]:
def bar_precipitacion():
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
    precTotal = {}
    mesDrop = widgets.Dropdown(
        options=meses,
        value=meses[len(meses)-1],
        description='Periodo:',
        disabled=False,
        layout={'width':'30%'},
        style={'description_width': '35%'}
    )
    barOutput = widgets.Output()
    s_max = 0
    for id in ids:
        precTotal[id] = []
        for mes in meses:
            if(mes != 'Anual'):
                mesIndex = meses.index(mes)
                f_df = bigDf[bigDf['momento'].dt.month == mesIndex+1]
                temp = f_df.query('ID_Estacion == '+id).dropna()["RR6_Valor"].sum()
                if(temp > s_max):
                    s_max = temp
                precTotal[id].append(temp)
            else:
                precTotal[id].append(bigDf.query('ID_Estacion == '+id).dropna()["RR6_Valor"].sum())

    def mes_dropdown(change):
        if change['type'] == 'change' and change['name'] == 'value':
            nonlocal precTotal, s_max
            mesIndex = meses.index(change['new'])
            values = []
            for id in ids:
                values.append(precTotal[id][mesIndex])
            
            replot(precTotal.keys(), values, change['new'])
    
    def replot(labels,values,tag):
        nonlocal s_max
        with barOutput:
            barOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            plt.bar(labels, values)
            if(tag!='Anual'):
                plt.ylim(0,s_max)
            plt.ylabel("Precipitacion acumulada (mm)")
            plt.xlabel("Estaciones")
            plt.suptitle("Precipitacion acumulada por estacion - Periodo "+tag+"\nAño 2023")
            plt.grid()
            plt.show()    

    values = []
    for id in ids:
        values.append(precTotal[id][len(meses) - 1])
            
    replot(precTotal.keys(), values, meses[len(meses) - 1])
    #print(precTotal)

    mesDrop.observe(mes_dropdown)

    H = widgets.HBox([barOutput, mesDrop], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de barra de precipitacion acumulada por cada estacion a lo largo del año:
        Permite ver cuales son los niveles de precipitacion durante periodos de un mes determinado y eL total del año. Generalmente, precipitaciones indican niveles de nubosidad considerables, los cuales podrian afectar la cantidad de energia generada por un panel solar si se quiere instalar uno.
        ''')

    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(V,textOut)
    return V

def plot_avgHr():
    colours = {}
    for id in ids:
        colours[id] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
    meses = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic"]
    meanDf = pd.DataFrame(columns=ids)
    err = True
    selected=ids
    
    for x in range(1,13):
        filtered_df = bigDf[bigDf['momento'].dt.month == x]
        temp = pd.DataFrame(columns=ids)
        for id in ids:
            est = filtered_df.query('ID_Estacion == '+id).dropna()
            #RadGInst (watt/m2)
            temp[id] = [est["HR_Valor"].mean()]
            temp[id+"_std"] = [est["HR_Valor"].std()]
        if(meanDf.size == 0):
            meanDf = pd.concat([temp], ignore_index=True)
        else:
            meanDf = pd.concat([meanDf,temp], ignore_index=True)

    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    errChk = widgets.Checkbox(description="Desviacion estandar?", value=err, indent=False, layout={'width':'90%'})
    plotOutput = widgets.Output()

    def errChange(change):
        nonlocal err, selected
        #El cambio de estado del checkbox dispara una serie de cambios de estado. No es un check de True <-> False simple.
        #Como yo lo entiendo: Lock -> Cambio de estado -> Lock -> Nuevo estado
        if(change['type'] == 'change' and change['name'] == 'value'):
            err = not err
            replot(selected, err)
    
    def checkboxes(_):
        nonlocal err, selected
        selected = [label for label,chk in zip(ids,chkbxs) if chk.value]
        replot(selected, err)
            
    def replot(selected, dv):
        with plotOutput:
            plotOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            for id in selected:
                if(dv):
                    plt.errorbar(meses, meanDf[id], meanDf[id+"_std"], color=colours[id], capsize=5, fmt="-o", elinewidth=1, label = id)
                else:
                    plt.plot(meses, meanDf[id], label = id, color=colours[id], marker='o') 
            plt.xlabel("2023")
            plt.ylabel("Humedad relativa del aire(%)")
            plt.grid()
            plt.ylim(0,100)
            plt.legend()
            plt.suptitle("Humedad relativa promedio mensual\nAño 2023")
            plt.show()

    replot(selected,err)

    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    errChk.observe(errChange)
        
    V_in = widgets.VBox(chkbxs)
    ac_chk = widgets.Accordion(children=(V_in,), titles=('Estaciones',), layout={'width':'90%'})
    V_in2 =  widgets.VBox([ac_chk,errChk])
    H = widgets.HBox([plotOutput,V_in2], layout={'border': '1px solid black'})
    #display(H)
    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de linea con desviacion estandar de la humedad relativa promedio mensual:
        Permite ver los niveles de humedad promedio que detecta cada estacion durante todos los meses. Tambien muestra que tan consistente es el valor promedio, visualizando una barra da error.
        Los niveles de humedad pueden indicar posible nubosidad, afectando la efectividad de un panel solar. Tambien puede afectar directamente en el panel solar en forma de agua acumulada en la superficie del panel por condensacion.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    return V


def scatter_tempHum():
    colours = {}
    s_ids = ids
    s_opt = "Ambos"
    for id in ids:
        colours[id+"_scatter"] = (round(random.uniform(0,1),3),round(random.uniform(0,1),3),round(random.uniform(0,1),3))
        colours[id+"_line"] = (colours[id+"_scatter"][2],colours[id+"_scatter"][1],colours[id+"_scatter"][0])
    chkbxs = [widgets.Checkbox(description=label, value=True, indent=False, layout={'width':'90%'}) for label in labels]
    scatterOutput = widgets.Output()


    def replot(selected_ids, option):
        lmDict = {}
        with scatterOutput:
            scatterOutput.clear_output(wait=True)
            plt.figure(figsize=(10,5))
            #display(sel_data)
            for id in selected_ids:
                filtered_df = bigDf.query('ID_Estacion == '+id)
                plt.scatter(filtered_df["Ts_Valor"], filtered_df["HR_Valor"], color=colours[id+"_scatter"], label = id+"_Dsp", s=3)
            plt.xlim(-15,35)
            #plt.ylim(0,35)
            plt.xlabel("Temperatura(C°)")
            plt.ylabel("Humedad relativa del aier(%)")
            plt.suptitle("Relacion de niveles de Humedad relativa del aire y temperatura\nAño 2023")
            plt.legend()
            plt.grid()
            plt.show()
    
    def checkboxes(_):
        nonlocal s_ids, s_opt
        s_ids = [label for label,chk in zip(labels,chkbxs) if chk.value]
        replot(s_ids, s_opt)
    
    for chk in chkbxs:
        chk.observe(checkboxes, 'value')

    replot(s_ids, s_opt)
    V_in = widgets.VBox(chkbxs, layout={'justify-content':'flex-start'})
    accordion = widgets.Accordion(children=(V_in,), titles=('Estaciones',))
    #V_in2 = widgets.VBox([accordion], layout={'width':'20%'})
    H = widgets.HBox([scatterOutput, accordion], layout={'border': '1px solid black'})

    textOut = widgets.Output(layout={'border': '1px solid black'})
    with textOut:
        print('''Grafico de dispersion de temperatura y humedad relativa del aire:
        Permite ver la relacion entre temperatura y la humedad del aire segun la estacion. Puede observarse una tendencia de disminucion de la humedad en cuanto la temperatura comienza a in
        Debido a que las estaciones estan situadas en diferentes sectores del pais, puede observarse a que niveles de temperatura la humedad comienza a disminuir en cada sector.
        ''')
    V = widgets.VBox([H,textOut], layout={'width':'33%'})
    #display(H,textOut)
    return V
    

#dim_lluvia()


In [112]:
def dim_lluvia():
    H = widgets.HBox([plot_avgHr(), scatter_tempHum(), bar_precipitacion()], layout={'height':'600px','width':'2600px'})

    display(H)
    
def dim_viento():
    H = widgets.HBox([boxPlot_viento(), scatter_vientoHum(), plotTimeSlider_viento()], layout={'height':'600px','width':'2600px'})

    display(H)

def dim_radiacion():
    H = widgets.HBox([pie_radiacionTotal(), scatter_tempRadiacion(), plotTimeSlider_rad()], layout={'height':'600px','width':'2600px'})

    display(H)

In [113]:
tabs = ["Viento", "Radiacion", "Agua"]

sub_tab=[widgets.Output() for i in range(len(tabs))]
tab = widgets.Tab(sub_tab)
for i in range (len(tabs)):
    tab.set_title(i,tabs[i])
    with sub_tab[i]:
        if(i == 0):
            dim_viento()
        elif(i == 1):
            dim_radiacion()
        else:
            dim_lluvia()
display(tab)